### Graph erstellen

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, LineString
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

def get_centroid(polygons):
    '''
    Fügt den Zentroid zu den Polygonen hinzu
    polygons: geladene Polygon-Shapefile-Daten
    '''
    polygons['centroid'] = polygons['geometry'].centroid
    return polygons

def get_closest_point(line, point):
    '''Ermittelt den nächstgelegenen Punkt auf der Linie zu einem Punkt'''
    closest_point = line.interpolate(line.project(point))
    return closest_point

def closest_points_buildings(polygons, lines):
    '''Ermittelt für jedes Polygon mit HU_calc > 0 den nächsten Punkt (Anschlusspunkt) auf dem Linien-Netzwerk und die street-ID'''
    
    # Erstellen Sie einen räumlichen Index für die Linien
    sindex = lines.sindex

    # Iteration über jeden Polygon-Zentroiden
    for index, row_p in polygons.iterrows():
        if row_p['HU_calc'] > 0:
            centroid = row_p['centroid']

            # Verwenden Sie den räumlichen Index, um die nächstgelegenen Linien zum Zentroiden zu erhalten
            possible_matches_index = list(sindex.nearest(centroid))
            possible_matches = lines.iloc[[i[0] for i in possible_matches_index]]

            # Finden Sie die Linie, die dem Zentroiden am nächsten liegt
            closest_line = possible_matches.geometry.distance(centroid).idxmin()

            # Ermitteln Sie den nächsten Punkt auf dieser Linie
            closest_point = get_closest_point(lines.at[closest_line, 'geometry'], centroid)

            polygons.at[index, 'Anschlusspunkt'] = closest_point
            polygons.at[index, 'street_id'] = int(closest_line)
    
    return polygons

def closest_points_sources(sources, lines):
    '''Ermittelt für jede Quelle den nächsten Punkt (Anschlusspunkt) auf dem Linien-Netzwerk und die street-ID'''
    # Iteration über jede Quelle und Suche des nächstgelegenen Punkts auf dem Linien-Netzwerk
    for index, row_s in sources.iterrows():

        # Initialisieren der Variablen für den minimalen Abstand und den nächstgelegenen Punkt
        min_distance = float('inf')
        closest_point = None
        source = row_s['geometry']

        # Iteration über jede Linie im Linien-Netzwerk
        for idx,row in lines.iterrows():
            line_coords = list(row['geometry'].coords)  # Liste der Punkte, aus denen die Linie besteht
            
            # Iteration über jeden Linienabschnitt, um den nächstgelegenen Punkt zu finden
            for i in range(1, len(line_coords)):
                start_point = Point(line_coords[i-1])
                end_point = Point(line_coords[i])
                line_segment = LineString([start_point, end_point])

                distance = line_segment.distance(source)

                if distance < min_distance:
                    min_distance = distance
                    closest_point = get_closest_point(LineString([start_point, end_point]), source)
                    id = idx
        sources.at[index, 'Anschlusspunkt'] = closest_point
        sources.at[index, 'street_id'] = int(id)
    return sources

def add_connection_to_streets(buildings, streets, sources):
    '''Anschlusspunkte in die Straßenlinien einfügen'''

    for df in [buildings, sources]:
        for index, row in df.iterrows():
            street_id = row['street_id']
            if not pd.isna(street_id):
                anschlusspunkt = row['Anschlusspunkt']
                line = streets['geometry'][street_id]
                line_coords = list(line.coords)

                insertion_position = None
                min_distance = float('inf')

                # Einfügeposition in der Linie finden
                for i in range(1, len(line_coords)):
                    segment = LineString([line_coords[i-1], line_coords[i]])
                    distance = segment.distance(anschlusspunkt)

                    if distance < min_distance:
                        min_distance = distance
                        insertion_position = i

                # Fügen Sie den ursprünglichen Anschlusspunkt in die Linienkoordinaten ein
                if (anschlusspunkt.x, anschlusspunkt.y) not in line_coords:
                    line_coords.insert(insertion_position, (anschlusspunkt.x, anschlusspunkt.y))
                    streets.at[street_id, 'geometry'] = LineString(line_coords)
    return streets

def create_street_network(streets):
    '''Graph (Straßen-Netzwerk) erstellen, Knoten und Kannten hinzufügen'''
    # Graph erstellen
    G = nx.Graph()

    # Knoten hinzufügen
    for idx, row in streets.iterrows():
        geom = row['geometry']
        line_coords = list(geom.coords)

        # Iteration über jeden Punkt auf der Linie
        for i in range(len(line_coords)):
            node = line_coords[i]
            G.add_node(node)

            # Verbindung zu vorherigem Punkt (außer beim ersten Punkt)
            if i > 0:
                prev_node = line_coords[i-1]
                G.add_edge(node, prev_node)
    return G

def connect_centroids(G, buildings):
    '''Fügt die Zentroiden der Gebäude zum Netz G hinzu'''
    for index, row in buildings.iterrows():
        centroid = row['centroid']
        closest_point = row['Anschlusspunkt']
        if not pd.isna(closest_point):
            G.add_edge(centroid.coords[0], (closest_point.x, closest_point.y))
    return G

def connect_source(G, sources):
    '''Fügt Energiequelle dem Netz G hinzu'''
    for index, row in sources.iterrows():
        source = row['geometry']
        closest_point = row['Anschlusspunkt']
        if not pd.isna(source):
            G.add_edge(source.coords[0], (closest_point.x, closest_point.y))
    return G

def add_attribute_length(G):
    # Kantenattribute hinzufügen
    for node1, node2 in G.edges():
        geom = LineString([node1, node2])
        G.edges[node1, node2]['length'] = geom.length
    return G

def graph(buildings, streets, sources):
    '''Baut das Netz(Graph) aus Gebäuden, Straßen und Quellen'''

    # Hinzufügen der Zentroide zu den Polygonen
    buildings = get_centroid(buildings)

    # Berechnung der nächstgelegenen Punkte für Gebäude und Quellen
    buildings = closest_points_buildings(buildings, streets)
    sources = closest_points_sources(sources, streets)

    # Anschlusspunkte in die Straßenlinien einfügen
    streets = add_connection_to_streets(buildings, streets, sources)
    
    # Graph (Straßennetz) erstellen 
    G = create_street_network(streets)

    # Zentroiden der Gebäude an Graph anschließen
    G = connect_centroids(G, buildings)

    # Quellen an Graph anschließen
    G = connect_source(G,sources) 

    # Attribute hinzufügen
    G = add_attribute_length(G)

    return G

def plot_G(G):

    # Koordinatensystem festlegen
    pos = {node: (node[0], node[1]) for node in G.nodes}

    # Graph anzeigen
    plt.figure(figsize=(50, 50))
    nx.draw_networkx(G, pos=pos, with_labels=False, font_size=6, node_size=3, node_color='blue', edge_color='gray')
    plt.show()

# Laden der Shapefiles
polygon_file = "Gebaeude.shp"
source_file = "test_source.shp"
lines_file = "Streets_adjusted.shp"

streets = gpd.read_file(lines_file)
buildings = gpd.read_file(polygon_file)
sources = gpd.read_file(source_file)

G = graph(buildings,streets,sources)
plot_G(G)